# Import libraries

In [47]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation,GlobalMaxPooling2D,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow.keras.backend as keras_backend

# Download and unzip data

In [2]:
!wget https://www.dropbox.com/s/t4pzwpvrzneb190/training_set.zip
!wget https://www.dropbox.com/s/i37jfni3d29raoc/test_set.zip

In [6]:
!unzip -o training_set.zip 
!unzip -o test_set.zip

# Parameters for the project

In [108]:
img_w,img_h= 150,150
training_dir = r"training_set/training_set"
validation_dir=r"test_set/test_set"
epochs= 20
batch_size=20


# Creating Image data generators with Augmentation


In [119]:
# 
train_datagen=ImageDataGenerator ( rescale=1. /255,shear_range =0.2,zoom_range=0.2,horizontal_flip =True)
test_datagen=ImageDataGenerator (rescale=1. /255)

train_generator =train_datagen.flow_from_directory(training_dir,target_size =(img_w,img_h), batch_size=batch_size,class_mode='binary',classes=['cats','dogs'])
validation_generator =test_datagen.flow_from_directory(validation_dir,target_size =(img_w,img_h), batch_size=batch_size,class_mode='binary')


In [120]:
if keras_backend.image_data_format()=='channels_first': 
    input_shape=(3, img_w, img_h)
else:
    input_shape=(img_w,img_h,3)

# Display data from each batch of data generators

In [121]:
plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in train_generator:
        # print(Y_batch)
        image = X_batch[0] #sampling first image from each batch
        if Y_batch[0] == 0.0:
            plt.xlabel("Cat")
        else:
            plt.xlabel("Dog")
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

# Pretrained model building

In [122]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model

In [123]:
pretrained_model= MobileNetV2(include_top=False,weights='imagenet',input_shape=input_shape)

In [128]:
print('no of layers:> ',len(pretrained_model.layers))

layer_list = [layers.name for layers in pretrained_model.layers]
# print(layer_list)
# print('Index of the first layer of block 7:>  ',layer_list.index("block7a_expand_conv"))

# make all the block till 352 as non-trainable
for layers in pretrained_model.layers:
    layers.trainable = False

# make all the block from 353 as trainable
# for layers in pretrained_model.layers:
#     if layers._keras_api_names[0] == 'keras.layers.BatchNormalization':
#         layers.trainable = False

last_layer = pretrained_model.get_layer('out_relu') #final layer
last_output = last_layer.output

# Adding the head to the pretrained model
x = GlobalMaxPooling2D()(last_output)
x = BatchNormalization()(x)
# x = Dropout(0.2, name="top_dropout")(x)
# x = Dense(256, activation="relu")(x)
# x = Dense(128, activation="relu")(x)
# x = Dense(64, activation="relu")(x)
# x = Dense(32, activation="relu")(x)
x = Dense(1, activation="sigmoid", name="pred")(x)
Mobilenet_v2_model = Model(pretrained_model.input, x)

# Mobilenet_v2_model.summary()


# Checking how many layers are trainable

In [129]:
# for layers in Efficientnet_model.layers:
#     print(layers._keras_api_names,"> ",layers.trainable)

# Compiling and fitting the model

In [130]:
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

Mobilenet_v2_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [141]:
training = Mobilenet_v2_model.fit(train_generator,
                   steps_per_epoch=100,epochs=50,
                   validation_data=validation_generator,
                       validation_steps=100,callbacks=callbacks)

# Visualization

In [142]:
import matplotlib.pyplot as plt
%matplotlib inline
# list all data in training
print(training.history.keys())
# summarize training for accuracy
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize traning for loss
plt.plot(training.history['loss'])
plt.plot(training.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Prediction

In [145]:
Mobilenet_v2_model.evaluate(validation_generator)

In [144]:
from tensorflow.keras.preprocessing import image
import numpy as np
img_pred = image.load_img("test_set/test_set/dogs/dog.4003.jpg",target_size=(150,150))

img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred, axis=0)

result = Efficientnet_model.predict(img_pred)
print(result)
if result[0][0]==1:
    prediction ="Dog"
else:
    prediction ="Cat"
print('Prediction: ',prediction)

img=mpimg.imread('test_set/test_set/dogs/dog.4006.jpg')
imgplot = plt.imshow(img)
plt.show()


# Explanations

* As a part of assignment 2, I went for pre-trained MobileNet-V2 model with the Top= False and added custom head to the model.
* Removed the drop_out layer as the model was underfitting because of the drop_out.
* Tried adding "softmax" instead of "sigmoid" activation function but that lead to decrease in accuracy.
* Increase in epoch didn't have much effect on the accuracy as the model reached it's peak accuracy and training stopped early.
* Updated from SGD optimizer to Adam which boosted the accuracy.

* **Finally achived the test accuracy of 95.79%**